In [1]:
import numpy as np
from random import shuffle
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

from vigenere_utils import *
import pandas as pd
import shutil, os

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.contrib import rnn, seq2seq

In [2]:
show_result('madlabrules', 'deeplearning')

Key: deeplearning
Plaintext: madlabrules
Encrytped: pehalfrlymf
Decrytped: madlabrules


In [3]:
logs_path = '/home/user/tensorflow_logs/vigenere_RNN_Telegram_test'
key = "madlab"
seq_length = 15

In [4]:
def get_random_sequence(ALPHA, seq_length):
        seq = ''.join(np.random.choice(a=list(ALPHA), replace=True, size=seq_length))
        return seq

In [5]:
def one_hot_encode(sequence, n_unique=len(ALPHA)):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[ALPHA.index(value)] = 1
		encoding.append(vector)
	return np.array(encoding)

In [6]:
np.random.seed(1616)
original_sequences = [get_random_sequence(ALPHA, seq_length) for i in range(2**20)]
np.random.seed(1616)
shuffle(original_sequences)

In [7]:
encrypted_sequences = [encrypt(key=key, plaintext=s) for s in original_sequences]

In [8]:
original_sequences_oh = [one_hot_encode(s) for s in original_sequences]
encrypted_sequences_oh = [one_hot_encode(s) for s in encrypted_sequences]
original_argmax = []
encrypted_argmax = []

In [9]:
for s in original_sequences_oh:
    t = []
    for c in s:
        t.extend([np.argmax(c)])
    original_argmax.append(t)

In [10]:
for s in encrypted_sequences_oh:
    t = []
    for c in s:
        t.extend([np.argmax(c)])
    encrypted_argmax.append(t)

In [11]:
batch_size = 10000
num_steps = seq_length
num_classes = len(ALPHA)
state_size = 256
num_layers = 3
learning_rate = 0.01
tf.reset_default_graph()

In [38]:
updater = Updater(token="312721681:AAEjt4EGxYAiRiemThOoccWQXgEHVXxvHCE")
dispatcher = updater.dispatcher
updater.start_polling()

In [39]:
def vigenere_RNN(bot, update):
     bot.send_message(chat_id=update.message.chat_id, \
     	text="Epoch: "+"{:.0f}".format(j)+" - Batch: "+"{:.0f}".format(i)+" - loss: "+"{:f}".format(training_loss_))

In [40]:
rnn_handler = CommandHandler('vigenere_RNN', vigenere_RNN)
dispatcher.add_handler(rnn_handler)

In [53]:
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

rnn_inputs = tf.one_hot(x, num_classes)

In [54]:
cell = tf.contrib.rnn.BasicRNNCell(state_size)
#cell = tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True)

In [55]:
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, dtype=tf.float32)

In [56]:
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
logits = tf.reshape(
            tf.matmul(tf.reshape(rnn_outputs, [-1, state_size]), W) + b,
            [batch_size, num_steps, num_classes])
predictions = tf.nn.softmax(logits)


In [57]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

In [58]:
tf.summary.scalar("error", total_loss)
# Create a summary to monitor accuracy tensor
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
    
merged_summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name rnn/basic_rnn_cell/kernel:0 is illegal; using rnn/basic_rnn_cell/kernel_0 instead.
INFO:tensorflow:Summary name rnn/basic_rnn_cell/bias:0 is illegal; using rnn/basic_rnn_cell/bias_0 instead.
INFO:tensorflow:Summary name softmax/W:0 is illegal; using softmax/W_0 instead.
INFO:tensorflow:Summary name softmax/b:0 is illegal; using softmax/b_0 instead.


In [59]:
def generate_batch(encrypted, original, batch_size):
    n_batches = len(encrypted)/batch_size
    for n in range(int(n_batches)):
        if((n+1)*batch_size+1 < len(encrypted_argmax)):
            yield(encrypted_argmax[n*batch_size:(n+1)*batch_size],\
                 original_argmax[n*batch_size:(n+1)*batch_size])
        else:
            yield(encrypted_argmax[n*batch_size:], \
                 original_argmax[n*batch_size:])

def gen_epoch(encrypted, original, n_epochs, batch_size):
    for e in range(n_epochs):
        yield(generate_batch(encrypted, original, batch_size))

In [60]:
if(os.path.exists(logs_path)):
    print("Removing logs")
    shutil.rmtree(logs_path)

Removing logs


In [ ]:
if(os.path.exists(logs_path)):
    print("Removing logs")
    shutil.rmtree(logs_path)

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph())

training_state = np.zeros((batch_size, state_size))

st=-1
for j, gen in enumerate(gen_epoch(encrypted_argmax, original_argmax, 5, batch_size)):
    print("Epoch")
    for i, (X, Y) in enumerate(gen):
        st+=1
        tr_losses, training_loss_, training_state, _, p, summary = \
                            sess.run([losses,
                                      total_loss,
                                      final_state,
                                      train_step,
                                     logits,
                                     merged_summary_op],
                                          feed_dict={x:X, y:Y})
        summary_writer.add_summary(summary, st)
        if(i % 1 == 0):
            print("Epoch: "+"{:.0f}".format(j)+" - Batch: "+"{:.0f}".format(i)+" - loss: "+"{:f}".format(training_loss_))
            print("input: "+''.join([ALPHA[i] for i in X[0]]))
            print("target: "+''.join([ALPHA[i] for i in Y[0]]))
            print("pred: "+''.join([ALPHA[np.argmax(i)] for i in p[0]]))

Epoch
Epoch: 0 - Batch: 0 - loss: 3.287793
input: pjahkixjduywujr
target: djxwkhljajyvijo
pred: xaappiktppkarlx
Epoch: 0 - Batch: 1 - loss: 3.438817
input: tuqsacglcgnvezx
target: hunhabulzvnuszu
pred: uxjvixowddajmvj
Epoch: 0 - Batch: 2 - loss: 3.459884
input: mcjwmjmtxivtekc
target: acglmiatuxvsskz
pred: jzfduzcuzyauyai
Epoch: 0 - Batch: 3 - loss: 3.674712
input: ovlddkzojvtvijt
target: cvisdjnogktuwjq
pred: vuuzuusluxuzuuu
Epoch: 0 - Batch: 4 - loss: 3.364608
input: ejfdkrwyufpxash
target: sjcskqkyrupwose
pred: sjxzsazskzskuxk
Epoch: 0 - Batch: 5 - loss: 4.116775
input: zpgqfmotefjovkz
target: npdfflctbujnjkw
pred: cujxtuwkuhvusvu
Epoch: 0 - Batch: 6 - loss: 3.900141
input: zxkamgrerctzazm
target: nxhpmffeortyozj
pred: cujaosrzsjosrks
Epoch: 0 - Batch: 7 - loss: 4.030994
input: ixinjmxvpdniflc
target: wxfcjllvmsnhtlz
pred: hujrjyreyrtnstn
Epoch: 0 - Batch: 8 - loss: 3.949982
input: vmhhbjmqkbqxrmq
target: jmewbiaqhqqwfmn
pred: uscwjrbyuznstzy
Epoch: 0 - Batch: 9 - loss: 4.507477
inp

Epoch: 0 - Batch: 80 - loss: 4.157909
input: padlgmonppdbmzu
target: daaaglcnmedaazr
pred: daeyuozgykuggyk
Epoch: 0 - Batch: 81 - loss: 4.231046
input: unecqvsuzyrmoeh
target: inbrquguwnrlcee
pred: inykooaykooanko
Epoch: 0 - Batch: 82 - loss: 4.354243
input: pmhlbjuqtqmjbaw
target: dmeabiiqqfmipat
pred: dahkooahkooahko
Epoch: 0 - Batch: 83 - loss: 4.091516
input: jbplfrbnappricv
target: xbmafqpnxepqwcs
pred: jamkoihmtoahmoo
Epoch: 0 - Batch: 84 - loss: 3.979213
input: bsgzqooqomsvzvh
target: psdoqncqlbsunve
pred: bnptoiamtoippvd
Epoch: 0 - Batch: 85 - loss: 4.366940
input: wlyembalkovtumj
target: klvtmaolhdvsimg
pred: wnptvipmeoippvi
Epoch: 0 - Batch: 86 - loss: 4.026967
input: esgwqrxfxqdynuz
target: ssdlqqlfufdxbuw
pred: snptfspmediptvi
Epoch: 0 - Batch: 87 - loss: 3.551466
input: ucqthqhwkyjcalj
target: icnihpvwhnjbolg
pred: icnviiztfspbefs
Epoch: 0 - Batch: 88 - loss: 4.367323
input: ldhbahfrgsymeuu
target: zdeqagtrdhylsur
pred: lcptfspbmisztfs
Epoch: 0 - Batch: 89 - loss: 4.008664

Epoch: 0 - Batch: 163 - loss: 4.322105
input: fmflsbglkowzpqn
target: tmcasaulhdwydqk
pred: tlpmlimqlmelpml
Epoch: 0 - Batch: 164 - loss: 3.847472
input: vogparfbookouyu
target: jodeaqtbldkniyr
pred: vlpmliiqlpelpul
Epoch: 0 - Batch: 165 - loss: 4.088629
input: fntfkppavbavvsh
target: tnqukodasqaujse
pred: tnpuliullpsliul
Epoch: 0 - Batch: 166 - loss: 3.732631
input: icqbcscojszchwy
target: wcnqcrqoghzbvwv
pred: wcklitniukiukli
Epoch: 0 - Batch: 167 - loss: 4.256865
input: nrapdyhvqosojcf
target: brxedxvvndsnxcc
pred: niukiuknitnduki
Epoch: 0 - Batch: 168 - loss: 3.683776
input: jksqzaxljierkeu
target: xkpfzzllgxeqyer
pred: jniukqdkqdxnduk
Epoch: 0 - Batch: 169 - loss: 3.561568
input: drfhahtrqmlbwcb
target: rrcwaghrnblakcy
pred: deqdxndxkqdkqcx
Epoch: 0 - Batch: 170 - loss: 3.943609
input: cxuonmgijaqhzxd
target: qxrdnluigpqgnxa
pred: ceqdendxkqdeqcx
Epoch: 0 - Batch: 171 - loss: 4.563114
input: eswtypqzvjbgqtm
target: sstiyoezsybfetj
pred: secxeqcxnzxrqde
Epoch: 0 - Batch: 172 - loss

Epoch: 0 - Batch: 242 - loss: 4.025645
input: iwiovdcyumnflfy
target: wwfdvcqyrbnezfv
pred: iwrmxqmlqhlyucm
Epoch: 0 - Batch: 243 - loss: 3.832275
input: zurxtaxeyrawksv
target: nuomtzlevgavyss
pred: nmxkmedzcmxkmld
Epoch: 0 - Batch: 244 - loss: 3.900182
input: xxbtnalvgppdsts
target: lxyinzzvdepcgtp
pred: xmzcmxktedzrmxk
Epoch: 0 - Batch: 245 - loss: 3.816556
input: wpjvwgoxqpqfixa
target: kpgkwfcxneqewxx
pred: wrdzcmxktedzrmx
Epoch: 0 - Batch: 246 - loss: 4.148170
input: uaabcevthwwcryv
target: iaxqcdjtelwbfys
pred: itegvcgadtegvcg
Epoch: 0 - Batch: 247 - loss: 3.505866
input: pcculkyzizvujdf
target: dczjljmzfovtxdc
pred: dqaitdgtdgaitdg
Epoch: 0 - Batch: 248 - loss: 4.060206
input: gaseuoewtvgbsbl
target: uaptunswqkgagbi
pred: uqphadqvibphadq
Epoch: 0 - Batch: 249 - loss: 4.141508
input: selrzrqrogczjxu
target: geigzqerlvcyxxr
pred: hqdhadgpildqydg
Epoch: 0 - Batch: 250 - loss: 4.263755
input: zllqselxeykztgc
target: nlifsdzxbnkyhgz
pred: zhydgpildqydgpi
Epoch: 0 - Batch: 251 - loss

Epoch: 0 - Batch: 319 - loss: 3.525105
input: uxkdxhzareubelc
target: ixhsxgnaotuaslz
pred: ijhvacyerjhvacy
Epoch: 0 - Batch: 320 - loss: 4.032834
input: nmhiyvylleatjvo
target: bmexyumlitasxvl
pred: bpcyarjhvamyarj
Epoch: 0 - Batch: 321 - loss: 3.995976
input: lyawuerqvjwouqr
target: zyxludfqsywniqo
pred: zlvaudaujhvauda
Epoch: 0 - Batch: 322 - loss: 3.912135
input: nqhomhsivkashfp
target: bqedmggiszarvfm
pred: btuxuvduvaudaud
Epoch: 0 - Batch: 323 - loss: 3.790547
input: gfcuqretjngwqto
target: ufzjqqstgcgvetl
pred: uwdudtudauduxtu
Epoch: 0 - Batch: 324 - loss: 4.345728
input: swzansumdfxuwyk
target: gwwpnrimauxtkyh
pred: gxtuduxtudaudux
Epoch: 0 - Batch: 325 - loss: 3.689463
input: kbscvkwpqievela
target: ybprvjkpnxeuslx
pred: yxlvdfdtudlxded
Epoch: 0 - Batch: 326 - loss: 4.038500
input: zenixlbqxgvsmtz
target: nekxxkpquvvratw
pred: zesyezysdesyuzy
Epoch: 0 - Batch: 327 - loss: 3.941256
input: ewgrhghlmjfbvqz
target: swdghfvljyfajqw
pred: swzekyekysdenye
Epoch: 0 - Batch: 328 - loss

...

Epoch: 0 - Batch: 2356 - loss: 3.796194
input: usngyvzuphzszzp
target: iskvyunumwzrnzm
pred: israrararararar
Epoch: 0 - Batch: 2357 - loss: 3.635747
input: ypfmtyebfwguohw
target: mpcbtxsbclgtcht
pred: mvaxaxaxaxaxaxa
Epoch: 0 - Batch: 2358 - loss: 3.252262
input: geaunjnxhgumveo
target: uexjnibxevuljel
pred: uxaxaxaxaxaxaxa
Epoch: 0 - Batch: 2359 - loss: 3.550787
input: ijxurnpxqbyefge
target: wjujrmdxnqydtgb
pred: wdududududududu
Epoch: 0 - Batch: 2360 - loss: 4.111979
input: npshkpnfqcpglut
target: bppwkobfnrpfzuq
pred: beueueueueueueu
Epoch: 0 - Batch: 2361 - loss: 3.360269
input: aysyrqzbqhsbmlv
target: oypnrpnbnwsaals
pred: ofrerererererer
Epoch: 0 - Batch: 2362 - loss: 3.751514
input: dibbtxqsuenhenw
target: riyqtwesrtngsnt
pred: rdrfrfrfrfrfrfr
Epoch: 0 - Batch: 2363 - loss: 3.768246
input: zfrcnezzejemidh
target: nforndnzbyelwde
pred: ngrfrfrfrfrfrfr
Epoch: 0 - Batch: 2364 - loss: 3.891037
input: yhnevwhuozwgwzs
target: mhktvvvulowfkzp
pred: mwtftftftftftft
Epoch: 0 - Batch: 2

Epoch: 0 - Batch: 2436 - loss: 3.748553
input: pdpmkydgfjpuxlt
target: ddmbkxrgcyptllq
pred: dkxfvfvfvfvfvfv
Epoch: 0 - Batch: 2437 - loss: 3.459205
input: nkxnljorvxknspx
target: bkuclicrsmkmgpu
pred: bhzfzfzfzfzfzfz
Epoch: 0 - Batch: 2438 - loss: 3.261480
input: oeeynxofcadocui
target: cebnnwcfzpdnquf
pred: ckzfzfzfzfzfzfz
Epoch: 0 - Batch: 2439 - loss: 3.405470
input: mwekavapyfgnhyx
target: awbzauopvugmvyu
pred: arzwzwzwzwzwzwz
Epoch: 0 - Batch: 2440 - loss: 3.263471
input: fnpfvnleianrcsy
target: tnmuvmzefpnqqsv
pred: tpzwzwzwzwzwzwz
Epoch: 0 - Batch: 2441 - loss: 3.754428
input: bykhjsfcuuelrxs
target: pyhwjrtcrjekfxp
pred: ppkmkmkmkmkmkmk
Epoch: 0 - Batch: 2442 - loss: 3.424795
input: mkyzgnzdtwwslbm
target: akvogmndqlwrzbj
pred: amqpvpvpvpvpvpv
Epoch: 0 - Batch: 2443 - loss: 4.136822
input: zjvvedzzsgpshlp
target: njskecnzpvprvlm
pred: npqufufufufufuf
Epoch: 0 - Batch: 2444 - loss: 4.017267
input: tgjguerrivgfizu
target: hggvudfrfkgewzr
pred: hnufufufufufufu
Epoch: 0 - Batch: 2

Epoch: 0 - Batch: 2515 - loss: 3.570660
input: fioqocmpueenrke
target: tilfobaprtemfkb
pred: tnibibibibibibi
Epoch: 0 - Batch: 2516 - loss: 3.967475
input: lekcvaifygommga
target: zehrvzwfvvolagx
pred: znububububububu
Epoch: 0 - Batch: 2517 - loss: 3.898461
input: uqlyplgeamvivxl
target: iqinpkuexbvhjxi
pred: iotutututututut
Epoch: 0 - Batch: 2518 - loss: 3.767902
input: yvllvmbzpavypio
target: mviavlpzmpvxdil
pred: mthihihihihihih
Epoch: 0 - Batch: 2519 - loss: 4.123872
input: kshzkkhpxcgcxdr
target: yseokjvpurgbldo
pred: yihihihihihihih
Epoch: 0 - Batch: 2520 - loss: 4.809584
input: ejrfbesoffjqglg
target: sjoubdgocujpuld
pred: soqzbibibibibib
Epoch: 0 - Batch: 2521 - loss: 4.247059
input: vffrfuwqudnjulx
target: jfcgftkqrsniilu
pred: jtbibibibibibib
Epoch: 0 - Batch: 2522 - loss: 3.748708
input: kqkyaikhtcbrtyd
target: yqhnahyhqrbqhya
pred: yibjbjbjbjbjbjb
Epoch: 0 - Batch: 2523 - loss: 3.666017
input: njfssfaquoagcfn
target: bjchseoqrdafqfk
pred: bpbpbpbpbpbpbpb
Epoch: 0 - Batch: 2

Epoch: 0 - Batch: 2592 - loss: 3.722681
input: xzzrtrlddhilgbe
target: lzwgtqzdawikubb
pred: lrerbrbrbrbrbrb
Epoch: 0 - Batch: 2593 - loss: 4.845747
input: dtsfyrkcndnrpzf
target: rtpuyqycksnqdzc
pred: rrerererererere
Epoch: 0 - Batch: 2594 - loss: 4.561990
input: hjgfhghesekaqbm
target: vjduhfveptkzebj
pred: vymqererererere
Epoch: 0 - Batch: 2595 - loss: 3.477035
input: fwsspnparcbxsht
target: twphpmdaorbwghq
pred: tqeriririririri
Epoch: 0 - Batch: 2596 - loss: 4.620255
input: eoaschextwomdge
target: soxhcgsxqlolrgb
pred: sbmqisisisisisi
Epoch: 0 - Batch: 2597 - loss: 3.562252
input: nrffkvxunlksymb
target: brcukulukakrmmy
pred: bwpspspspspspsp
Epoch: 0 - Batch: 2598 - loss: 3.992181
input: nyughdguhrbovjv
target: byrvhcuuegbnjjs
pred: bwpwpwpwpwpwpwp
Epoch: 0 - Batch: 2599 - loss: 3.632883
input: wnnkltsrgxanlys
target: knkzlsgrdmamzyp
pred: kgptktktktktktk
Epoch: 0 - Batch: 2600 - loss: 3.580887
input: eccentnvugsjbdb
target: scztnsbvrvsipdy
pred: sbduktktktktktk
Epoch: 0 - Batch: 2

Epoch: 0 - Batch: 2668 - loss: 3.624321
input: nfftqrsoimwyaps
target: bfciqqgofbwxopp
pred: byititititititi
Epoch: 0 - Batch: 2669 - loss: 3.661321
input: skffivwfcbgvyaa
target: gkcuiukfzqgumax
pred: gkototototototo
Epoch: 0 - Batch: 2670 - loss: 3.295794
input: hnjefzukazcvejl
target: vngtfyikxocusji
pred: vjauotototototo
Epoch: 0 - Batch: 2671 - loss: 3.541612
input: xwbiwnhmdygcbph
target: lwyxwmvmangbppe
pred: lzototototototo
Epoch: 0 - Batch: 2672 - loss: 3.935329
input: emzyjhqknntivkn
target: smwnjgekkcthjkk
pred: sjgugugugugugug
Epoch: 0 - Batch: 2673 - loss: 3.584487
input: lotokfbfsrobdto
target: zoqdkepfpgoartl
pred: zugugugugugugug
Epoch: 0 - Batch: 2674 - loss: 3.100913
input: vxdkvurkvluvyfk
target: jxazvtfksauumfh
pred: jxgugugugugugug
Epoch: 0 - Batch: 2675 - loss: 3.367409
input: sqbkavasdtputwp
target: gqyzauosaipthwm
pred: gxlulululululul
Epoch: 0 - Batch: 2676 - loss: 3.215206
input: gahszoinpkphjlq
target: uaehznwnmzpgxln
pred: uolulululululul
Epoch: 0 - Batch: 2

In [26]:
print("input: "+''.join([ALPHA[i] for i in X[0]]))
print("target: "+''.join([ALPHA[i] for i in Y[0]]))
print("pred: "+''.join([ALPHA[np.argmax(i)] for i in p[0]]))
print("pred_enc: "+encrypt(key, ''.join([ALPHA[np.argmax(i)] for i in p[0]])))

input: vyckjzgnphcoddz
target: jyzzjyunmwcnrdw
pred: jyzzjyunmwcnrdw
pred_enc: vyckjzgnphcoddz


In [23]:
print(encrypted_sequences[i*batch_size])
print(original_sequences[i*batch_size])
print(encrypt(key, original_sequences[i*batch_size]))

vyckjzgnphcoddz
jyzzjyunmwcnrdw
vyckjzgnphcoddz
